In [1]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

In [2]:
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
drive_path = "/content/drive/MyDrive"

In [4]:
batch_size = 4
lr = 1e-4
epochs = 25
height = 64
width = 64

In [5]:
dataset_path = os.path.join(drive_path,"dataset_u-net","non-aug")
files_dir = os.path.join(drive_path, "Colab Notebooks","files","non-aug")
model_file = os.path.join(files_dir,"unet-non-aug.h5")
log_file = os.path.join(files_dir,"log-non-aug.csv")

In [6]:
def create_dir(path):
  if not os.path.exists(path):
    os.makedirs(path)

In [7]:
 create_dir(files_dir)

In [8]:
def conv_block(inputs, num_filters):
  x = Conv2D(num_filters, 3, padding="same")(inputs)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  x = Conv2D(num_filters, 3 ,padding = "same")(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  return x

In [9]:
def encoder_block(inputs, num_filters):
  x = conv_block(inputs, num_filters)
  p = MaxPool2D((2, 2))(x)
  return x,p

In [10]:
def decoder_block(inputs, skip, num_filters):
  x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding ="same")(inputs)
  x = Concatenate()([x, skip])
  x = conv_block(x, num_filters)
  return x

In [11]:
def build_unet(input_shape):
  inputs = Input(input_shape)
  s1,p1 = encoder_block(inputs, 64)
  s2,p2 = encoder_block(p1, 128)
  s3,p3 = encoder_block(p2, 256)
  s4,p4 = encoder_block(p3, 512)

  b1 = conv_block(p4, 1024)

  d1 = decoder_block(b1,s4, 512)
  d2 = decoder_block(d1,s3, 256)
  d3 = decoder_block(d2,s2, 128)
  d4 = decoder_block(d3,s1, 64)

  outputs = Conv2D(1, 1, padding="same", activation = "sigmoid")(d4)

  model = Model(inputs, outputs, name="UNET")
  return model

In [12]:
def load_data(path):
  train_x = sorted(glob(os.path.join(path, "train", "original", "*")))
  train_y = sorted(glob(os.path.join(path, "train", "mask", "*")))

  valid_x = sorted(glob(os.path.join(path, "validation", "original", "*")))
  valid_y = sorted(glob(os.path.join(path, "validation", "mask", "*")))

  return (train_x, train_y), (valid_x, valid_y)

In [13]:
def read_image(path):
  path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_COLOR)
  x = x/255.0
  return x


In [14]:
def read_mask(path):
  path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  x = x/255.0
  x = np.expand_dims(x, axis =-1)
  return x


In [15]:
def tf_parse(x,y):
  def _parse(x,y):
    x = read_image(x)
    y = read_mask(y)
    return x, y

  x, y = tf.numpy_function(_parse, [x,y], [tf.float64,tf.float64])
  x.set_shape([height, width, 3])
  y.set_shape([height, width, 1])

  return x,y

In [16]:
def tf_dataset(x,y, batch=8):
  dataset = tf.data.Dataset.from_tensor_slices((x,y))
  dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
  dataset = dataset.batch(batch)
  dataset = dataset.prefetch(tf.data.AUTOTUNE)
  return dataset

In [17]:
(train_x, train_y), (valid_x, valid_y) = load_data(dataset_path)
print(f"train: {len(train_x)} - {len(train_y)}")
print(f"validation: {len(valid_x)} - {len(valid_y)}")

train: 1256 - 1256
validation: 157 - 157


In [18]:
train_dataset = tf_dataset(train_x,train_y, batch=8)
valid_dataset = tf_dataset(valid_x,valid_y, batch=8)

In [19]:
for x,y in valid_dataset:
  print(x.shape,y.shape)

(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(8, 64, 64, 3) (8, 64, 64, 1)
(5, 64, 64, 3) (5, 64, 64, 1)


In [20]:
input_shape = (height,width,3)
model = build_unet(input_shape)

In [21]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 64)   1792        ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 64, 64, 64)   0           ['batch_normalization[0][0]'] 

In [22]:
opt = tf.keras.optimizers.Adam(lr)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["acc"])

In [23]:
callbacks = [
    ModelCheckpoint(model_file, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=4),
    CSVLogger(log_file),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
]

In [24]:
model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/25
157/157 [==============================] - ETA: 0s - loss: 0.5721 - acc: 0.7664
Epoch 1: val_loss improved from inf to 0.52318, saving model to /content/drive/MyDrive/Colab Notebooks/files/non-aug/unet-non-aug.h5
157/157 [==============================] - 908s 6s/step - loss: 0.5721 - acc: 0.7664 - val_loss: 0.5232 - val_acc: 0.7624 - lr: 1.0000e-04
Epoch 2/25
157/157 [==============================] - ETA: 0s - loss: 0.4014 - acc: 0.8202
Epoch 2: val_loss improved from 0.52318 to 0.41036, saving model to /content/drive/MyDrive/Colab Notebooks/files/non-aug/unet-non-aug.h5
157/157 [==============================] - 13s 81ms/step - loss: 0.4014 - acc: 0.8202 - val_loss: 0.4104 - val_acc: 0.8273 - lr: 1.0000e-04
Epoch 3/25
157/157 [==============================] - ETA: 0s - loss: 0.3658 - acc: 0.8341
Epoch 3: val_loss improved from 0.41036 to 0.37059, saving model to /content/drive/MyDrive/Colab Notebooks/files/non-aug/unet-non-aug.h5
157/157 [==============================] 